In [2]:
!pip install pgmpy

In [0]:
'''from google.colab import files
uploaded = files.upload()'''

'from google.colab import files\nuploaded = files.upload()'

In [3]:
! git clone https://github.com/kumarankit1996/Dataset.git

fatal: destination path 'Dataset' already exists and is not an empty directory.


In [4]:
!ls Dataset/AML_Proj2/

 15features.csv			  feature_Unseen_weights_v1.h5
 31features_seen_train.csv	 'Icon'$'\r'
 31_Seen_Predict_Val.csv	  seen-dataset
 31_Seen_Train.csv		  seen_predict_feature.csv
 31_Seen_Val.csv		  seen_weights_v1_e10k.h5
 31_shuffled_Predict_Val.csv	  shuffled-dataset
 31_shuffled_Train.csv		  shuffled_predict_feature.csv
 31_shuffled_Val.csv		  shuffled_weights_v1_e10k.h5
 31_unSeen_Predict_Val.csv	  unseen-dataset
 31_unSeen_Train.csv		  unseen_predict_feature.csv
 31_unSeen_Val.csv		  Unseen_weights_v1_1.h5
 feature_seen_weights_v1.h5	  Unseen_weights_v1_e10k.h5
 feature_shuffled_weights_v1.h5


# Import Library

In [0]:
import numpy as np
import pandas as pd
import math
import io
from keras.layers import concatenate
import random
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Reshape, UpSampling2D,Flatten, Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.layers import Dense
from keras import optimizers
import cv2
from PIL import Image
import os
from google.colab.patches import cv2_imshow
import math
import timeit

# Import PGM library

In [0]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import K2Score
from pgmpy.inference import VariableElimination
from timeit import default_timer as timer
from pgmpy.estimators import ExhaustiveSearch
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BayesianEstimator

# Reading datasets

In [0]:
# read csv dataset
features=pd.read_csv("Dataset/AML_Proj2/15features.csv")
#removing unwanted columns
#features=features.drop(["imagename"],axis=1)
features.set_index('imagename',inplace=True)
all_nodes=['pen_pressure1', 'letter_spacing1', 'size1', 'dimension1', 'is_lowercase1',
       'is_continuous1', 'slantness1', 'tilt1', 'entry_stroke_a1', 'staff_of_a1',
       'formation_n1', 'staff_of_d1', 'exit_stroke_d1', 'word_formation1',
       'constancy1','pen_pressure2', 'letter_spacing2', 'size2', 'dimension2', 'is_lowercase2',
       'is_continuous2', 'slantness2', 'tilt2', 'entry_stroke_a2', 'staff_of_a2',
       'formation_n2', 'staff_of_d2', 'exit_stroke_d2', 'word_formation2',
       'constancy2','label']
feature_nodes=['pen_pressure1', 'letter_spacing1', 'size1', 'dimension1', 'is_lowercase1',
       'is_continuous1', 'slantness1', 'tilt1', 'entry_stroke_a1', 'staff_of_a1',
       'formation_n1', 'staff_of_d1', 'exit_stroke_d1', 'word_formation1',
       'constancy1','pen_pressure2', 'letter_spacing2', 'size2', 'dimension2', 'is_lowercase2',
       'is_continuous2', 'slantness2', 'tilt2', 'entry_stroke_a2', 'staff_of_a2',
       'formation_n2', 'staff_of_d2', 'exit_stroke_d2', 'word_formation2',
       'constancy2']

In [0]:
important=np.zeros((15,15))
# finding correlation of features
c=features.corr()

In [8]:
for i in range (0,15):
    for j in range(0,15):
        if c.iloc[i,j]<0.1:
            # if less than threshold then less correlation
            important[i][j]=0
        else:
            if i!=j:
              important[i][j]=1
df=pd.DataFrame.from_records(important,columns=feature_nodes[:15])
df

,pen_pressure1,letter_spacing1,size1,dimension1,is_lowercase1,is_continuous1,slantness1,tilt1,entry_stroke_a1,staff_of_a1,formation_n1,staff_of_d1,exit_stroke_d1,word_formation1,constancy1
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


# Task 2

# Bayesian Models

 Model 1

In [0]:
structure1_2=[('pen_pressure2','letter_spacing2'),('dimension2', 'size2'),('size2','slantness2'),('is_lowercase2', 'staff_of_a2'),('is_continuous2', 'exit_stroke_d2'),('is_continuous2', 'entry_stroke_a2'),('is_continuous2', 'is_lowercase2'),('slantness2', 'tilt2'),('formation_n2', 'constancy2'), ('formation_n2', 'word_formation2'), ('formation_n2', 'dimension2'),('formation_n2', 'size2'), ('formation_n2', 'staff_of_a2'),('staff_of_d2', 'is_continuous2'), ('staff_of_d2', 'exit_stroke_d2'), ('staff_of_d2', 'is_lowercase2'),('word_formation2', 'dimension2'), ('word_formation2', 'staff_of_a2'), ('word_formation2', 'size2'),('word_formation2', 'constancy2'), ('constancy2', 'staff_of_a2'),('constancy2', 'dimension2'),('constancy2', 'size2'),('pen_pressure2', 'exit_stroke_d2')]
structure1_1=[('pen_pressure1','letter_spacing1'),('dimension1', 'size1'),('size1','slantness1'),('is_lowercase1', 'staff_of_a1'),('is_continuous1', 'exit_stroke_d1'),('is_continuous1', 'entry_stroke_a1'),('is_continuous1', 'is_lowercase1'),('slantness1', 'tilt1'),('formation_n1', 'constancy1'), ('formation_n1', 'word_formation1'), ('formation_n1', 'dimension1'),('formation_n1', 'size1'), ('formation_n1', 'staff_of_a1'),('staff_of_d1', 'is_continuous1'), ('staff_of_d1', 'exit_stroke_d1'), ('staff_of_d1', 'is_lowercase1'),('word_formation1', 'dimension1'), ('word_formation1', 'staff_of_a1'), ('word_formation1', 'size1'),('word_formation1', 'constancy1'), ('constancy1', 'staff_of_a1'),('constancy1', 'dimension1'),('constancy1', 'size1'),('pen_pressure1', 'exit_stroke_d1')]

Model 2

In [0]:
structure2_1=[('pen_pressure1', 'is_lowercase1'), ('pen_pressure1', 'letter_spacing1'), ('size1', 'slantness1'), ('size1', 'pen_pressure1'), ('size1', 'staff_of_d1'), ('size1', 'letter_spacing1'), ('size1', 'exit_stroke_d1'), ('size1', 'entry_stroke_a1'), ('dimension1', 'size1'), ('dimension1', 'is_continuous1'), ('dimension1', 'slantness1'), ('dimension1', 'pen_pressure1'), ('is_lowercase1', 'staff_of_a1'), ('is_lowercase1', 'exit_stroke_d1'), ('is_continuous1', 'exit_stroke_d1'), ('is_continuous1', 'letter_spacing1'), ('is_continuous1', 'entry_stroke_a1'), ('is_continuous1', 'staff_of_a1'), ('is_continuous1', 'is_lowercase1'), ('slantness1', 'is_continuous1'), ('slantness1', 'tilt1'), ('entry_stroke_a1', 'pen_pressure1'), ('formation_n1', 'constancy1'), ('formation_n1', 'word_formation1'), ('formation_n1', 'dimension1'), ('formation_n1', 'staff_of_d1'), ('formation_n1', 'is_continuous1'), ('formation_n1', 'size1'), ('formation_n1', 'staff_of_a1'), ('staff_of_d1', 'is_continuous1'), ('staff_of_d1', 'exit_stroke_d1'), ('staff_of_d1', 'is_lowercase1'), ('staff_of_d1', 'slantness1'), ('staff_of_d1', 'entry_stroke_a1'), ('word_formation1', 'dimension1'), ('word_formation1', 'staff_of_a1'), ('word_formation1', 'size1'), ('word_formation1', 'staff_of_d1'), ('word_formation1', 'constancy1'), ('constancy1', 'staff_of_a1'), ('constancy1', 'letter_spacing1'), ('constancy1', 'dimension1')]
structure2_2=[('pen_pressure2', 'is_lowercase2'), ('pen_pressure2', 'letter_spacing2'), ('size2', 'slantness2'), ('size2', 'pen_pressure2'), ('size2', 'staff_of_d2'), ('size2', 'letter_spacing2'), ('size2', 'exit_stroke_d2'), ('size2', 'entry_stroke_a2'), ('dimension2', 'size2'), ('dimension2', 'is_continuous2'), ('dimension2', 'slantness2'), ('dimension2', 'pen_pressure2'), ('is_lowercase2', 'staff_of_a2'), ('is_lowercase2', 'exit_stroke_d2'), ('is_continuous2', 'exit_stroke_d2'), ('is_continuous2', 'letter_spacing2'), ('is_continuous2', 'entry_stroke_a2'), ('is_continuous2', 'staff_of_a2'), ('is_continuous2', 'is_lowercase2'), ('slantness2', 'is_continuous2'), ('slantness2', 'tilt2'), ('entry_stroke_a2', 'pen_pressure2'), ('formation_n2', 'constancy2'), ('formation_n2', 'word_formation2'), ('formation_n2', 'dimension2'), ('formation_n2', 'staff_of_d2'), ('formation_n2', 'is_continuous2'), ('formation_n2', 'size2'), ('formation_n2', 'staff_of_a2'), ('staff_of_d2', 'is_continuous2'), ('staff_of_d2', 'exit_stroke_d2'), ('staff_of_d2', 'is_lowercase2'), ('staff_of_d2', 'slantness2'), ('staff_of_d2', 'entry_stroke_a2'), ('word_formation2', 'dimension2'), ('word_formation2', 'staff_of_a2'), ('word_formation2', 'size2'), ('word_formation2', 'staff_of_d2'), ('word_formation2', 'constancy2'), ('constancy2', 'staff_of_a2'), ('constancy2', 'letter_spacing2'), ('constancy2', 'dimension2')]

Model 3 

In [0]:
structure3_1=[('pen_pressure1', 'is_lowercase1'),  ('size1', 'slantness1'), ('size1', 'pen_pressure1'),  ('size1', 'letter_spacing1'), ('size1', 'exit_stroke_d1'), ('size1', 'entry_stroke_a1'), ('dimension1', 'size1'), ('dimension1', 'is_continuous1'),  ('dimension1', 'pen_pressure1'), ('is_lowercase1', 'staff_of_a1'), ('is_lowercase1', 'exit_stroke_d1'), ('is_continuous1', 'exit_stroke_d1'),  ('is_continuous1', 'entry_stroke_a1'), ('is_continuous1', 'staff_of_a1'), ('is_continuous1', 'is_lowercase1'), ('slantness1', 'is_continuous1'), ('slantness1', 'tilt1'),  ('formation_n1', 'constancy1'), ('formation_n1', 'word_formation1'), ('formation_n1', 'dimension1'), ('formation_n1', 'staff_of_d1'), ('formation_n1', 'is_continuous1'), ('formation_n1', 'size1'), ('formation_n1', 'staff_of_a1'), ('staff_of_d1', 'is_continuous1'), ('staff_of_d1', 'exit_stroke_d1'), ('staff_of_d1', 'is_lowercase1'), ('staff_of_d1', 'slantness1'), ('staff_of_d1', 'entry_stroke_a1'), ('word_formation1', 'dimension1'), ('word_formation1', 'staff_of_a1'), ('word_formation1', 'size1'), ('word_formation1', 'staff_of_d1'), ('word_formation1', 'constancy1'), ('constancy1', 'staff_of_a1'), ('constancy1', 'letter_spacing1'), ('constancy1', 'dimension1')]
structure3_2=[('pen_pressure2', 'is_lowercase2'),  ('size2', 'slantness2'), ('size2', 'pen_pressure2'),  ('size2', 'letter_spacing2'), ('size2', 'exit_stroke_d2'), ('size2', 'entry_stroke_a2'), ('dimension2', 'size2'), ('dimension2', 'is_continuous2'),  ('dimension2', 'pen_pressure2'), ('is_lowercase2', 'staff_of_a2'), ('is_lowercase2', 'exit_stroke_d2'), ('is_continuous2', 'exit_stroke_d2'),  ('is_continuous2', 'entry_stroke_a2'), ('is_continuous2', 'staff_of_a2'), ('is_continuous2', 'is_lowercase2'), ('slantness2', 'is_continuous2'), ('slantness2', 'tilt2'),  ('formation_n2', 'constancy2'), ('formation_n2', 'word_formation2'), ('formation_n2', 'dimension2'), ('formation_n2', 'staff_of_d2'), ('formation_n2', 'is_continuous2'), ('formation_n2', 'size2'), ('formation_n2', 'staff_of_a2'), ('staff_of_d2', 'is_continuous2'), ('staff_of_d2', 'exit_stroke_d2'), ('staff_of_d2', 'is_lowercase2'), ('staff_of_d2', 'slantness2'), ('staff_of_d2', 'entry_stroke_a2'), ('word_formation2', 'dimension2'), ('word_formation2', 'staff_of_a2'), ('word_formation2', 'size2'), ('word_formation2', 'staff_of_d2'), ('word_formation2', 'constancy2'), ('constancy2', 'staff_of_a2'), ('constancy2', 'letter_spacing2'), ('constancy2', 'dimension2')]

Model 4

In [0]:
structure4_1=[('pen_pressure1', 'is_lowercase1'), ('pen_pressure1', 'letter_spacing1'), ('size1', 'slantness1'),  ('size1', 'staff_of_d1'), ('size1', 'letter_spacing1'), ('size1', 'exit_stroke_d1'), ('size1', 'entry_stroke_a1'), ('dimension1', 'size1'), ('dimension1', 'is_continuous1'), ('dimension1', 'slantness1'), ('dimension1', 'pen_pressure1'),  ('is_lowercase1', 'exit_stroke_d1'), ('is_continuous1', 'exit_stroke_d1'), ('is_continuous1', 'letter_spacing1'), ('is_continuous1', 'entry_stroke_a1'), ('is_continuous1', 'staff_of_a1'), ('is_continuous1', 'is_lowercase1'), ('slantness1', 'is_continuous1'), ('slantness1', 'tilt1'), ('entry_stroke_a1', 'pen_pressure1'), ('formation_n1', 'constancy1'), ('formation_n1', 'word_formation1'), ('formation_n1', 'dimension1'), ('formation_n1', 'staff_of_d1'), ('formation_n1', 'is_continuous1'), ('formation_n1', 'size1'), ('formation_n1', 'staff_of_a1'), ('staff_of_d1', 'is_continuous1'), ('staff_of_d1', 'exit_stroke_d1'), ('staff_of_d1', 'is_lowercase1'), ('staff_of_d1', 'slantness1'), ('staff_of_d1', 'entry_stroke_a1'), ('word_formation1', 'dimension1'), ('word_formation1', 'staff_of_a1'), ('word_formation1', 'size1'), ('word_formation1', 'staff_of_d1'), ('word_formation1', 'constancy1'), ('constancy1', 'staff_of_a1'), ('constancy1', 'letter_spacing1'), ('constancy1', 'dimension1')]
structure4_2=[('pen_pressure2', 'is_lowercase2'), ('pen_pressure2', 'letter_spacing2'), ('size2', 'slantness2'),  ('size2', 'staff_of_d2'), ('size2', 'letter_spacing2'), ('size2', 'exit_stroke_d2'), ('size2', 'entry_stroke_a2'), ('dimension2', 'size2'), ('dimension2', 'is_continuous2'), ('dimension2', 'slantness2'), ('dimension2', 'pen_pressure2'),  ('is_lowercase2', 'exit_stroke_d2'), ('is_continuous2', 'exit_stroke_d2'), ('is_continuous2', 'letter_spacing2'), ('is_continuous2', 'entry_stroke_a2'), ('is_continuous2', 'staff_of_a2'), ('is_continuous2', 'is_lowercase2'), ('slantness2', 'is_continuous2'), ('slantness2', 'tilt2'), ('entry_stroke_a2', 'pen_pressure2'), ('formation_n2', 'constancy2'), ('formation_n2', 'word_formation2'), ('formation_n2', 'dimension2'), ('formation_n2', 'staff_of_d2'), ('formation_n2', 'is_continuous2'), ('formation_n2', 'size2'), ('formation_n2', 'staff_of_a2'), ('staff_of_d2', 'is_continuous2'), ('staff_of_d2', 'exit_stroke_d2'), ('staff_of_d2', 'is_lowercase2'), ('staff_of_d2', 'slantness2'), ('staff_of_d2', 'entry_stroke_a2'), ('word_formation2', 'dimension2'), ('word_formation2', 'staff_of_a2'), ('word_formation2', 'size2'), ('word_formation2', 'staff_of_d2'), ('word_formation2', 'constancy2'), ('constancy2', 'staff_of_a2'), ('constancy2', 'letter_spacing2'), ('constancy2', 'dimension2')]

In [0]:
# find the k2 of the given data
k2=K2Score(seen_train)
# Perform Hill Climb Search on the dataset
#hc=HillClimbSearch(features,scoring_method=k2)

In [0]:
# Predict the best model
#best_model=hc.estimate()
#print(best_model.edges())

In [0]:
def create_model(Train_data,s):
  if s==1:
    structure1=structure1_1
    structure2=structure1_2
  elif s==2:
    structure1=structure2_1
    structure2=structure2_2
  elif s==3:
    structure1=structure3_1
    structure2=structure3_2
  elif s==4:
    structure1=structure4_1
    structure2=structure4_2
  # Create Bayesian Model
  model=BayesianModel()
  # add edges to bayesian model
  model.add_edges_from(structure1)
  # replicating structure
  model.add_edges_from(structure2)
  # adding validation node
  model.add_edges_from([('pen_pressure1','label'),('pen_pressure2','label'),('tilt1','label'),('tilt2','label'),
                        ('staff_of_d1','label'),('staff_of_d2','label'),('formation_n1','label'),('formation_n2','label'),
                        ('exit_stroke_d1','label'),('exit_stroke_d2','label'),('is_continuous1','label'),('is_continuous2','label')])
  #Train bayesian model on Train data
  model.fit(Train_data)
  print('K2 Score:'+str(k2.score(model)))
  return model

# Test the model

In [0]:
def predict(model,data):
  '''total=len(data)
  predict_data=data.copy()
  predict_data.drop('label',axis=1,inplace=True)
  predict_data=predict_data.iloc[:total,:]
  predict_data=predict_data-1
  y=model.predict(predict_data)
  #predict_data
  label=data.iloc[:total,30].copy()
  accu=0
  for i, j in zip(y.values,label.values):
    if i==j:
      accu+=1'''
  inference=VariableElimination(model)
  accu=0
  data=data.sample(frac=1).reset_index(drop=True)
  lenght=data.shape[0]
  for row in range(0,lenght):
    actual=data.iloc[row,30]
    #### Predict right
    #print('predict right')
    # actual labels
    label=data.iloc[row,15:30].copy()
    # feature of left image
    all_feature=data.iloc[row,0:15].copy()
    all_feature=all_feature.to_dict()
    # subtracting 1 from all 15 features 
    for e in feature_nodes[:15]:
      all_feature[e]=all_feature[e]-1
    # label=0
    all_feature['label']=0
    #print('label 0:')
    label0=1
    #start Time
    #start = timer()
    phi_query_right_0=inference.query(feature_nodes[15:30],evidence=all_feature)#['label']
    for e in feature_nodes[15:30]:
      #print(phi_query[e])
      label0=label0*phi_query_right_0[e].values[label[e]-1]
    # label =1
    # label=0
    #print('label 1:')
    label1=1
    all_feature['label']=1
    #start Time
    #start = timer()
    phi_query_right_1=inference.query(feature_nodes[15:30],evidence=all_feature)#['label']
    for e in feature_nodes[15:30]:
      #print(phi_query[e])
      label1=label1*phi_query_right_1[e].values[label[e]-1]
    #print(all_feature)
    d1=(label0/label1)
    d1=math.log(d1)
    #if d1>=0:
      #predicted=0
    #else:
      #predicted=1
    #print(predicted,actual)
    #if predicted==actual:
      #accu+=1
  #print('accu:'+str(accu/lenght))
  
  
    #### Predict left
    print('predict left')
    # actual labels
    label=data.iloc[row,0:15].copy()
    # feature of left image
    all_feature=data.iloc[row,15:30].copy()
    all_feature=all_feature.to_dict()
    # subtracting 1 from all 15 features 
    for e in feature_nodes[15:30]:
      all_feature[e]=all_feature[e]-1
    # label=0
    all_feature['label']=0
    print('label 0:')
    label0=1
    #start Time
    #start = timer()
    phi_query_left_0=inference.query(feature_nodes[0:15],evidence=all_feature)#['label']
    for e in feature_nodes[0:15]:
      #print(phi_query[e])
      label0=label0*phi_query_left_0[e].values[label[e]-1]
    # label =1
    # label=0
    print('label 1:')
    label1=1
    all_feature['label']=1
    #start Time
    #start = timer()
    phi_query_left_1=inference.query(feature_nodes[0:15],evidence=all_feature)#['label']
    for e in feature_nodes[0:15]:
      #print(phi_query[e])
      label1=label1*phi_query_left_1[e].values[label[e]-1]
    d2=(label0/label1)
    d2=math.log(d2)
    d=(d1+d2)/2
    if d>=0:
      predicted=0
    else:
      predicted=1
    print(predicted,actual)
    if predicted==actual:
      accu+=1
  print('accu:'+str(accu))
  

# Concatinating dataset

In [0]:
def join_features(Train,name):
  #loop for number of rows in dataset
  v=[]
  for i in range(0,len(Train.index)):
    try:
      #extract features for left
      x=features.loc[Train.iloc[i,0],:]
      #extract features for right
      y=features.loc[Train.iloc[i,1],:]
      #extract label
      w=Train.iloc[i,2]
      #concat all 3 side by side
      z=np.hstack([x,y,w])
      # add values to list
      v.append(z)
    except:
      continue
  len(v)
  # convert into dataframe
  v=pd.DataFrame(v)
  # rename columns to dataframe
  v.columns=all_nodes
  # write into csv
  v.to_csv(name+".csv",sep=",",index=False)
  #files.download(name+".csv")

# SeenDataset

In [0]:
#read dataset
Train_seen_data=pd.read_csv("Dataset/AML_Proj2/seen-dataset/dataset_seen_training_siamese.csv")
# drop the unwanted column
#Train_data=Train_data.drop("number",axis=1)
#Train_data=Train_data.sample(frac=1).reset_index(drop=True)


#Train_seen_data.set_index('number',inplace=True)
#join_features(Train_seen_data,"31_Seen_Train")

#read dataset
Val_seen_data=pd.read_csv("Dataset/AML_Proj2/seen-dataset/dataset_seen_validation_siamese.csv")
# drop the unwanted column
#Val_data=Val_data.drop("number",axis=1)
#Val_data=Val_data.sample(frac=1).reset_index(drop=True)

#Val_seen_data.set_index('number',inplace=True)
#join_features(Train_seen_data,"31_Seen_Val")



In [50]:
len(Val_seen_data)

906

In [35]:
seen_train=pd.read_csv("Dataset/AML_Proj2/31_Seen_Train.csv")
seen_val=pd.read_csv("Dataset/AML_Proj2/31_Seen_Val.csv")
len(seen_val)

894

In [52]:

start = timeit.default_timer()
model=create_model(seen_train,1)

stop = timeit.default_timer()

print('Time: ', stop - start) 

K2 Score:-2256601.812071786
Time:  375.6305282039975


In [54]:
predict(model,seen_val)

accu: 60.4


# Unseen Dataset

In [0]:
#read dataset
Train_unseen_data=pd.read_csv("Dataset/AML_Proj2/unseen-dataset/dataset_seen_training_siamese.csv")
# drop the unwanted column
#Train_data=Train_data.drop("number",axis=1)
#Train_data=Train_data.sample(frac=1).reset_index(drop=True)


#Train_unseen_data.set_index('number',inplace=True)
#join_features(Train_unseen_data,"31_unSeen_Train")

#read dataset
Val_unseen_data=pd.read_csv("Dataset/AML_Proj2/unseen-dataset/dataset_seen_validation_siamese.csv")
# drop the unwanted column
#Val_data=Val_data.drop("number",axis=1)
#Val_data=Val_data.sample(frac=1).reset_index(drop=True)

#Val_unseen_data.set_index('number',inplace=True)
#join_features(Train_unseen_data,"31_unSeen_Val")



In [0]:
unseen_train=pd.read_csv("Dataset/AML_Proj2/31_unSeen_Train.csv")
unseen_val=pd.read_csv("Dataset/AML_Proj2/31_unSeen_Val.csv")


In [53]:

start = timeit.default_timer()

model=create_model(unseen_train,1)

stop = timeit.default_timer()

print('Time: ', stop - start)

K2 Score:-2256601.812071786
Time:  374.97740566300126


In [55]:
predict(model,unseen_val)

accu: 54.3


# Shuffled Dataset

In [0]:
#read dataset
Train_shuffled_data=pd.read_csv("Dataset/AML_Proj2/shuffled-dataset/dataset_seen_training_siamese.csv")
# drop the unwanted column
#Train_data=Train_data.drop("number",axis=1)
#Train_data=Train_data.sample(frac=1).reset_index(drop=True)

#Train_shuffled_data.set_index('number',inplace=True)
#join_features(Train_shuffled_data,"31_shuffled_Train")

#read dataset
Val_shuffled_data=pd.read_csv("Dataset/AML_Proj2/shuffled-dataset/dataset_seen_validation_siamese.csv")
# drop the unwanted column
#Val_data=Val_data.drop("number",axis=1)
#Val_data=Val_data.sample(frac=1).reset_index(drop=True)

#Val_shuffled_data.set_index('number',inplace=True)
#join_features(Train_shuffled_data,"31_shuffled_Val")


In [0]:
shuffled_train=pd.read_csv("Dataset/AML_Proj2/31_shuffled_Train.csv")
shuffled_val=pd.read_csv("Dataset/AML_Proj2/31_shuffled_Val.csv")


In [51]:

start = timeit.default_timer()
model=create_model(shuffled_train,4)

stop = timeit.default_timer()

print('Time: ', stop - start)

K2 Score:-2069727.6393613645
Time:  382.2711251569999


In [56]:
predict(model,shuffled_val)

accu: 55.7


# Task 3

# Siamese Network

In [0]:
imDim = 64
input_shape  = (imDim,imDim,1)
inp_img = Input(shape = (imDim,imDim,1), name = 'ImageInput')
model = inp_img

#     model = Input(shape=(imDim,imDim,1))
#     model.add(Input(shape = (imDim,imDim,1), name = 'FeatureNet_ImageInput'))
model = Conv2D(32,kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='valid')(model)

#####normalization
#model= Dropout(0.25)(model)

#     model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model = MaxPooling2D((2,2), padding='valid')(model)
model = Conv2D(64, (3, 3), activation='relu',padding='valid')(model)
#     model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model = MaxPooling2D((2,2),padding='valid')(model)

#####normalization
#model= BatchNormalization()(model)

#     model.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
model = Conv2D(128, (3, 3), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)
#     model.add(Conv2D(1, (3, 3), activation='relu',padding='same'))
#     model.add(Conv2D(2, (3, 3), activation='relu',padding='same'))

model = Conv2D(256, (1, 1), activation='relu',padding='valid')(model)
model = MaxPooling2D((2,2),padding='valid')(model)

model = Conv2D(64, (1, 1), activation='relu',padding='valid')(model)
# model = MaxPooling2D((2,2),padding='valid')(model)
model = Flatten()(model)

# img_in = np.array((-1,imDim,imDim,1), dtype='float32')
# img_in = tf.placeholder(shape=(imDim,imDim,1), dtype='float32')

feat = Model(inputs=[inp_img], outputs=[model],name = 'Feat_Model')
feat.summary()

left_img = Input(shape = (imDim,imDim,1), name = 'left_img')
right_img = Input(shape = (imDim,imDim,1), name = 'right_img')

left_feats = feat(left_img)
right_feats = feat(right_img)
merged_feats = concatenate([left_feats, right_feats], name = 'concat_feats')
merged_feats = Dense(1024, activation = 'linear')(merged_feats)
merged_feats=Dropout(0.5)(merged_feats)
#merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(4, activation = 'linear')(merged_feats)
merged_feats=Dropout(0.2)(merged_feats)
#merged_feats = BatchNormalization()(merged_feats)
merged_feats = Activation('relu')(merged_feats)
merged_feats = Dense(1, activation = 'sigmoid')(merged_feats)
similarity_model = Model(inputs = [left_img, right_img], outputs = [merged_feats], name = 'Similarity_Model')
similarity_model.summary()


In [0]:
similarity_model.compile(optimizer=optimizers.adadelta(lr=0.001),loss="binary_crossentropy",metrics=['accuracy'])

In [0]:
def image_dataset(data):
  Train_dataset={}
  Val_dataset={}
  #getting inside directory
  imgs=os.listdir("Dataset/AML_Proj2/"+data+"/TrainingSet/")
  #looping over all the images in the directory
  for name in imgs:
    img=cv2.imread("Dataset/AML_Proj2/"+data+"/TrainingSet/"+str(name),0)
    if img is not None:
      img=(255-img)/255
      Train_dataset[name]=img
  imgs=os.listdir("Dataset/AML_Proj2/"+data+"/ValidationSet/")
  #looping over all the images in the directory
  for name in imgs:
    img=cv2.imread("Dataset/AML_Proj2/"+data+"/ValidationSet/"+str(name),0)
    if img is not None:
      img=(255-img)/255
      Val_dataset[name]=img
  return Train_dataset,Val_dataset

# Image Dataset

In [0]:
Train_seen_images,Val_seen_images=image_dataset("seen-dataset")
Train_unseen_images,Val_unseen_images=image_dataset("unseen-dataset")
Train_shuffled_images,Val_shuffled_images=image_dataset("shuffled-dataset")

# Data Generation

In [0]:
def get_batches(start,end,batch_size,s):
  w=h=64
  if s=='seen_train':
    X=Train_seen_data
    dataset=Train_seen_images
  elif s=="seen_val":
    X=Val_seen_data
    dataset=Val_seen_images  
  elif s=="unseen_train":
    X=Train_unseen_data
    dataset=Train_unseen_images
  elif s=="unseen_val":
    X=Val_unseen_data
    dataset=Val_unseen_images
  elif s=="shuffled_train":
    X=Train_shuffled_data
    dataset=Train_shuffled_images
  elif s=="shuffled_val":
    X=Val_shuffled_data
    dataset=Val_shuffled_images
  X=X.sample(frac=1).reset_index(drop=True)
  # create list for pair images
  pairs=[np.zeros((batch_size,h,w,1)) for i in range(2)]
  # target of the pair
  targets=np.zeros((batch_size))
  # find min value for end
  end=min(len(Train_seen_data),end)
  # loop for each batch
  for i,j in zip(range(start,end),range(batch_size)):
    try:
      # storing left image in pair
      pairs[0][j,:,:,:]=dataset[X.iloc[i,0]].reshape(w,h,1)
      # storing right image in pair
      pairs[1][j,:,:,:]=dataset[X.iloc[i,1]].reshape(w,h,1)
      # storing target
      targets[j]=X.iloc[i,2]
    except:
      continue
  #print(pairs[0][0])
  return pairs,targets

In [0]:
def generate(batch_size,s):
  start=0
  end=batch_size
  while True:
    pairs, targets=get_batches(start,end,batch_size,s)
    yield(pairs,targets)
    start=end
    end=end+batch_size

# Seen Evaluation

In [0]:
batch_size=64
steps_per_epoch=math.ceil(len(Train_seen_data)/batch_size)
vsteps_per_epoch=math.ceil(len(Val_seen_data)/batch_size)
similarity_model.fit_generator(generate(batch_size,"seen_train"),epochs=10,steps_per_epoch=steps_per_epoch,validation_data=generate(batch_size,"seen_val"),validation_steps=vsteps_per_epoch)

# Unseen Evaluation

In [0]:
batch_size=64
steps_per_epoch=math.ceil(len(Train_seen_data)/batch_size)
vsteps_per_epoch=math.ceil(len(Val_seen_data)/batch_size)
similarity_model.fit_generator(generate(batch_size,"unseen_train"),epochs=10,steps_per_epoch=steps_per_epoch,validation_data=generate(batch_size,"unseen_val"),validation_steps=vsteps_per_epoch)

#  Shuffled Evaluation

In [0]:
batch_size=64
steps_per_epoch=math.ceil(len(Train_seen_data)/batch_size)
vsteps_per_epoch=math.ceil(len(Val_seen_data)/batch_size)
similarity_model.fit_generator(generate(batch_size,"shuffled_train"),epochs=10,steps_per_epoch=steps_per_epoch,validation_data=generate(batch_size,"shuffled_val"),validation_steps=vsteps_per_epoch)